<a href="https://colab.research.google.com/github/rtikyani/CS634_IBM_Fairness_Pipeline/blob/master/CS634_Project1_Team3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tutorial summary of the pipeline in a markdown file:

# [**Summary.md**](https://github.com/rtikyani/CS634_IBM_Fairness_Pipeline/blob/master/README.md)



# Execute the credit decision pipeline that is detecting age bias and removing using the reweighting algorithm and explain the findings:

In [ ]:
#Part 2

#install all necessary libraries and modules
!pip install aif360
!pip install IPython
!pip install BlackBoxAuditing
!pip install wget


#Retrieve Necessary Files 
import os
os.chdir('/usr/local/lib/python3.6/dist-packages/aif360/data/raw/german')
!wget -q	https://archive.ics.uci.edu/ml/machine-learning-databases/statlog/german/german.data
!wget -q	https://archive.ics.uci.edu/ml/machine-learning-databases/statlog/german/german.doc


#import the Methods used from aif360
from aif360.datasets import GermanDataset
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.algorithms.preprocessing import Reweighing

from IPython.display import Markdown, display

#Loading dataset
dataset_orig = GermanDataset(
    protected_attribute_names=['age'],           # this dataset also contains protected
                                                 # attribute for "sex" which we do not
                                                 # consider in this evaluation
    privileged_classes=[lambda x: x >= 25],      # age >=25 is considered privileged
    features_to_drop=['personal_status', 'sex'] # ignore sex-related attributes
)

dataset_orig_train, dataset_orig_test = dataset_orig.split([0.7], shuffle=True)
privileged_groups = [{'age': 1}]
unprivileged_groups = [{'age': 0}]

#Metric for original data set
metric_orig_train = BinaryLabelDatasetMetric(dataset_orig_train, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
display(Markdown("#### Original training dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_orig_train.mean_difference())

#Applying Reweighing Pre-Processing method to mitigate the bias

#Train with and transform the original training data
RW = Reweighing(unprivileged_groups=unprivileged_groups,
                privileged_groups=privileged_groups)
dataset_transf_train = RW.fit_transform(dataset_orig_train)

# Metric with the transformed training data

metric_transf_train = BinaryLabelDatasetMetric(dataset_transf_train, 
                                               unprivileged_groups=unprivileged_groups,
                                               privileged_groups=privileged_groups)
display(Markdown("#### Transformed training dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_transf_train.mean_difference())

# Findings:

[OUTPUT]: 

**Original training dataset**: Difference in mean outcomes between unprivileged and privileged groups = -0.169905 

**Transformed training dataset**: Difference in mean outcomes between unprivileged and privileged groups = 0.000000

**Explanation:**
The credit decision pipeline is an example of pre-processing mitigation, because the bias is mitigated before the creation of the model. After executing the credit decision pipeline on the training dataset to detect bias in the protected attribute, age, it was determined that the privileged group (aged 25+ years) had a higher percentage of favorable results over the unprivileged group (under 25 years old) by 17%. This was determined by the mean_difference method on the BinaryLabelDataset class, which calculates the percentage of favorable outcomes for the privileged group, and subtracts it from the percentage of favorable outcomes for the unprivileged group. Because the bias was removed before the creation of the model.

Once the age bias was identified (that the privileged group was getting 17% more positive outcomes from the training dataset), the reweighing algorithm was used to transform the training dataset to have more equity in positive outcomes on age for both the privileged and unprivileged groups. To determine the efficacy in removing bias of the transformation, the mean_difference method was again used on the transformed data, resulting in a difference in mean outcomes of 0. When comparing the mean outcomes from the original training dataset with that of the transformed training dataset, one can see from the difference in mean outcome values, -17% and 0% respectively, that the transformation effectively mitigated the 17% age bias to 0%.



# **Using [Disparate Impact Remover](https://aif360.readthedocs.io/en/latest/modules/generated/aif360.algorithms.preprocessing.DisparateImpactRemover.html#aif360.algorithms.preprocessing.DisparateImpactRemover)** for detecting and removing bias.


In [ ]:
#Part 3

#install all necessary libraries and modules
!pip install aif360
!pip install IPython
!pip install BlackBoxAuditing
!pip install wget


#Retrieve Necessary Files 
import os
os.chdir('/usr/local/lib/python3.6/dist-packages/aif360/data/raw/german')
!wget -q	https://archive.ics.uci.edu/ml/machine-learning-databases/statlog/german/german.data
!wget -q	https://archive.ics.uci.edu/ml/machine-learning-databases/statlog/german/german.doc


#import the Methods used from aif360
from aif360.datasets import GermanDataset
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.algorithms.preprocessing import Reweighing, DisparateImpactRemover

from IPython.display import Markdown, display

#Loading dataset
dataset_orig = GermanDataset(
    protected_attribute_names=['age'],           # this dataset also contains protected
                                                 # attribute for "sex" which we do not
                                                 # consider in this evaluation
    privileged_classes=[lambda x: x >= 25],      # age >=25 is considered privileged
    features_to_drop=['personal_status', 'sex'] # ignore sex-related attributes
)

dataset_orig_train, dataset_orig_test = dataset_orig.split([0.7], shuffle=True)
privileged_groups = [{'age': 1}]
unprivileged_groups = [{'age': 0}]

#Metric for original data set
metric_orig_train = BinaryLabelDatasetMetric(dataset_orig_train, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
display(Markdown("#### Original training dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_orig_train.mean_difference())


#Using Disparate Impact Remover PreProcessor Method  

#repair_level - Repair amount
#0.0 = no repair 
#1.0 is full repair.
DIR = DisparateImpactRemover(repair_level = 1.0)

#Run a repairer on the non-protected features and return the transformed dataset.
dataset_transf_DIR = DIR.fit_transform(dataset_orig_train)


dataset_transf_DIR_train, dataset_transf_DR_test, = dataset_transf_DIR.split([0.7], shuffle=True)

metric_DIR_train = BinaryLabelDatasetMetric(dataset_transf_DIR_train, unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups)

display(Markdown("#### Transformed training dataset using Disparate Impact Remover PreProcessor Method"))

print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_DIR_train.mean_difference())


# Findings using Disparate Impact Remover

The alternate method used was Disparate Impact Remover method. This is a different Pre-Processing technique which falls in the aif360 library. 

**Results:**

**Original training dataset:**
Difference in mean outcomes between unprivileged and privileged groups = -0.169905

**Transformed training dataset using Disparate Impact Remover PreProcessor Method:**
Difference in mean outcomes between unprivileged and privileged groups = -0.177184

Using the Disparate Impact Remover method, the difference in mean outcome between unprivileged and privileged groups was -0.177184, which is unfortunately a **greater** difference than using the Reweighing method and even the original dataset. 

This demonstrates that although multiple techniques can be utilized for PreProcessing data, not all will be optimal.The reason being that the different methods have unique algorithmic approaches. Like the Reweighing method, the DIR method aims to increases group fairness but does this while prioritizing preserving rank-ordering within groups. In this case and dataset, this doesn't help as the labels and protected attributes aren't modified when the dataset is transformed. On the other hand, the Reweighing technique calculates the weight of each combination of a group and label to ensure fairness. 

Since rank order within groups in this dataset is based on age, the protected attribute and also the source of the bias itself, our results could be indicative of the preservation of the rank-order throughout the transformation. Mitigating the age bias was our goal, but because rank-order, which is determined by age, was preserved throughout the transformation, we actually see an increase in the mean difference of outcomes.

For this dataset, the Reweighing method proved to be optimal as it transformed the dataset and mitigated the fairness to reach a difference in mean outcome of 0.00. If we were to do this again, we believe Optimized Pre-Processing would be the better alternate method, keeping the same protected attribute of age. This is especially apparent considering the size of this dataset. In hindsight, we see why DIR and LFR methods would be inefficient in mitigating fairness with this dataset. 

 
 


